In [10]:
from newsapi import NewsApiClient
import json
import pandas as pd
import numpy as np
import re
import datetime

In [2]:
# /v2/everything
#all_articles = newsapi.get_everything(q='bitcoin',
#                                          from_param='2020-12-01',
#                                          to='2020-12-24',
#                                          language='en',
#                                          sort_by='relevancy',
#                                          page=2)

In [3]:
# # # # NEWS API AUTHENTICATER # # # #
class NewsAuthenticator():

    def authenticate_news_api(self):
        auth = NewsApiClient(api_key='cc3f4e3191f149658f3922e9c47ec1ad')
        return auth

In [4]:
# # # # NEWS API STREAMER # # # #
class ArticleStreamer():
    """
    Class for streaming and processing articles daily
    """
    def __init__(self):
        self.news_authenticator = NewsAuthenticator()    

    def stream_articles(self, query, from_param, to):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        newsapi = self.news_authenticator.authenticate_news_api()
        all_articles = newsapi.get_everything(q=query,
                                          from_param= from_param,
                                          to= to,
                                          language='en')
        
        return all_articles

In [67]:
class ArticleCleaner():
    """
    Functionality for importing and cleaning news articles
    """
    def import_json(self, fetched_json_file):
        # Import json and normalize data
        df = pd.read_json(fetched_json_file)
        norm_articles = pd.json_normalize(df['articles'])
        
        return norm_articles

    def clean_article(self, article):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", article).split())
    
    def create_article(self, articles):
        article = articles[['author', 'title', 'description', 'url', 'source.name', 'publishedAt']]
        article['publishedAt'] = pd.to_datetime(article['publishedAt'])
        article.sort_values(by='publishedAt',inplace=True)
        article = article.reset_index()
        article = article.drop(['index'], axis=1)
        
        outlet = articles[['source.id', 'source.name']]
        
        return article, outlet

In [68]:
# /v2/top-headlines
#top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                          language='en',
#                                          category='business',
#                                          country='us')

In [69]:
# /v2/sources
#sources = newsapi.get_sources()

In [72]:
if __name__ == '__main__':
    fetched_json_file = './news_articles.json'
    query = 'cryptocurrency'
    from_param = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    to = datetime.date.today().strftime("%Y-%m-%d")
    
    article_cleaner = ArticleCleaner()
    article_streamer = ArticleStreamer()
    
    #all_articles = article_streamer.stream_articles(query, from_param, to)
    
    # Export json
    with open(fetched_json_file, 'w') as f:
        json.dump(all_articles, f)
    
    article_data = article_cleaner.import_json(fetched_json_file)
    final_news, outlets = article_cleaner.create_article(article_data)
    
    clean_title = np.array([article_cleaner.clean_article(article) for article in articles['title']])
    clean_desc = np.array([article_cleaner.clean_article(article) for article in articles['description']])
    
    final_news['clean_title'] = clean_title
    final_news['clean_description'] = clean_desc
    
    #final_news.to_csv(r'.\cleaned_articles_crypto.csv', index=False)
    #outlets.to_csv(r'.\outlets.csv', index=False)

<ipython-input-67-024b783b8439>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article['publishedAt'] = pd.to_datetime(article['publishedAt'])
<ipython-input-67-024b783b8439>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  article.sort_values(by='publishedAt',inplace=True)


In [71]:

articles

,author,title,description,url,source.name,publishedAt,clean_title,clean_description
0,Kevin Reynolds,Ethereum’s Ether Cryptocurrency Sets New All-T...,"The price of ether (ETH), the native cryptocur...",https://www.coindesk.com/index.php?p=569000,CoinDesk,2021-01-25 00:39:19+00:00,Ethereum s Ether Cryptocurrency Sets New All T...,The price of ether ETH the native cryptocurren...
1,"Gavin Butler, Julian Morgans",Police Raid Homes Following Shutdown of World’...,Less than a fortnight since DarkMarket was tak...,https://www.vice.com/en/article/7k9d3e/police-...,Vice News,2021-01-25 05:44:37+00:00,Police Raid Homes Following Shutdown of World ...,Less than a fortnight since DarkMarket was tak...
2,Anna Baydakova,Russian Public Officials Banned From Holding C...,Russia's Ministry of Labor has told governmet ...,https://www.coindesk.com/index.php?p=569026,CoinDesk,2021-01-25 11:02:58+00:00,Russian Public Officials Banned From Holding C...,Russia s Ministry of Labor has told governmet ...
3,matrix-org,Matrix.org Foundation accounts filing overdue ...,I wasn&#39;t sure if this is the right spot to...,https://github.com/matrix-org/matrix-doc/issue...,Github.com,2021-01-25 11:09:00+00:00,Matrix org Foundation accounts filing overdue ...,I wasn 39 t sure if this is the right spot to ...
4,Omkar Godbole,Big Investors Stacked up Ether as Price Rose t...,Strong hands look to be backing the recent ral...,https://www.coindesk.com/big-investors-stacked...,CoinDesk,2021-01-25 11:29:14+00:00,Big Investors Stacked up Ether as Price Rose t...,Strong hands look to be backing the recent ral...
5,newsfeedback@fool.com (Sean Williams),Bitcoin Is Now in a Bear Market -- Get Used to It,The world's largest cryptocurrency may be ente...,https://www.fool.com/investing/2021/01/25/bitc...,Motley Fool,2021-01-25 11:36:00+00:00,Bitcoin Is Now in a Bear Market Get Used to It,The world s largest cryptocurrency may be ente...
6,"Jeffrey Dastin, Sheila Dang, Anna Irrera","FOCUS-Online merchants linked to QAnon down, b...","Since last year, entrepreneur Dustin Krieger h...",https://www.reuters.com/article/usa-trump-qano...,Reuters,2021-01-25 12:00:00+00:00,FOCUS Online merchants linked to QAnon down bu...,Since last year entrepreneur Dustin Krieger ha...
7,Imran Tariq,How Millennials Are Changing Stock Investing,The thrifty nature of the nation's largest gen...,https://www.entrepreneur.com/article/362499,Entrepreneur,2021-01-25 13:00:00+00:00,How Millennials Are Changing Stock Investing,The thrifty nature of the nation s largest gen...
8,Danny Nelson,Crypto Miner Marathon Patent Group Buys $150M ...,Cryptocurrency mining company Marathon Patent ...,https://www.coindesk.com/miner-marathon-buys-b...,CoinDesk,2021-01-25 13:00:57+00:00,Crypto Miner Marathon Patent Group Buys 150M i...,Cryptocurrency mining company Marathon Patent ...
9,Jitendra Parashar,BlackBerry (TSX:BB) Stock: Time to Sell? - Yah...,<ol><li>BlackBerry (TSX:BB) Stock: Time to Sel...,https://ca.finance.yahoo.com/news/blackberry-t...,Yahoo Entertainment,2021-01-25 14:06:00+00:00,BlackBerry TSX BB Stock Time to Sell Yahoo Mov...,ol li BlackBerry TSX BB Stock Time to Sell Yah...


In [73]:
# Import BDay to determine business day's dates
from pandas.tseries.offsets import BDay

# to get the business day for which particular news headline should be used to make trade calls
def get_trade_open(date):
    curr_date_open = pd.to_datetime(date).floor('d').replace(hour=13,minute=30) - BDay(0)
    curr_date_close = pd.to_datetime(date).floor('d').replace(hour=20,minute=0) - BDay(0)
    
    prev_date_close = (curr_date_open - BDay()).replace(hour=20,minute=0)
    next_date_open = (curr_date_close + BDay()).replace(hour=13,minute=30)
    
    if ((pd.to_datetime(date)>=prev_date_close) & (pd.to_datetime(date)<curr_date_open)):
        return curr_date_open
    elif ((pd.to_datetime(date)>=curr_date_close) & (pd.to_datetime(date)<next_date_open)):
        return next_date_open
    else:
        return None
      
 # Apply the above function to get the trading time for each news headline
final_news["trading_time"] = final_news["publishedAt"].apply(get_trade_open)

final_news = final_news[pd.notnull(final_news['trading_time'])]
final_news['Date'] = pd.to_datetime(pd.to_datetime(final_news['trading_time']).dt.date)

In [74]:
final_news

,author,title,description,url,source.name,publishedAt,clean_title,clean_description,trading_time,Date
0,Kevin Reynolds,Ethereum’s Ether Cryptocurrency Sets New All-T...,"The price of ether (ETH), the native cryptocur...",https://www.coindesk.com/index.php?p=569000,CoinDesk,2021-01-25 00:39:19+00:00,Ethereum s Ether Cryptocurrency Sets New All T...,The price of ether ETH the native cryptocurren...,2021-01-25 13:30:00+00:00,2021-01-25
1,"Gavin Butler, Julian Morgans",Police Raid Homes Following Shutdown of World’...,Less than a fortnight since DarkMarket was tak...,https://www.vice.com/en/article/7k9d3e/police-...,Vice News,2021-01-25 05:44:37+00:00,Police Raid Homes Following Shutdown of World ...,Less than a fortnight since DarkMarket was tak...,2021-01-25 13:30:00+00:00,2021-01-25
2,Anna Baydakova,Russian Public Officials Banned From Holding C...,Russia's Ministry of Labor has told governmet ...,https://www.coindesk.com/index.php?p=569026,CoinDesk,2021-01-25 11:02:58+00:00,Russian Public Officials Banned From Holding C...,Russia s Ministry of Labor has told governmet ...,2021-01-25 13:30:00+00:00,2021-01-25
3,matrix-org,Matrix.org Foundation accounts filing overdue ...,I wasn&#39;t sure if this is the right spot to...,https://github.com/matrix-org/matrix-doc/issue...,Github.com,2021-01-25 11:09:00+00:00,Matrix org Foundation accounts filing overdue ...,I wasn 39 t sure if this is the right spot to ...,2021-01-25 13:30:00+00:00,2021-01-25
4,Omkar Godbole,Big Investors Stacked up Ether as Price Rose t...,Strong hands look to be backing the recent ral...,https://www.coindesk.com/big-investors-stacked...,CoinDesk,2021-01-25 11:29:14+00:00,Big Investors Stacked up Ether as Price Rose t...,Strong hands look to be backing the recent ral...,2021-01-25 13:30:00+00:00,2021-01-25
5,newsfeedback@fool.com (Sean Williams),Bitcoin Is Now in a Bear Market -- Get Used to It,The world's largest cryptocurrency may be ente...,https://www.fool.com/investing/2021/01/25/bitc...,Motley Fool,2021-01-25 11:36:00+00:00,Bitcoin Is Now in a Bear Market Get Used to It,The world s largest cryptocurrency may be ente...,2021-01-25 13:30:00+00:00,2021-01-25
6,"Jeffrey Dastin, Sheila Dang, Anna Irrera","FOCUS-Online merchants linked to QAnon down, b...","Since last year, entrepreneur Dustin Krieger h...",https://www.reuters.com/article/usa-trump-qano...,Reuters,2021-01-25 12:00:00+00:00,FOCUS Online merchants linked to QAnon down bu...,Since last year entrepreneur Dustin Krieger ha...,2021-01-25 13:30:00+00:00,2021-01-25
7,Imran Tariq,How Millennials Are Changing Stock Investing,The thrifty nature of the nation's largest gen...,https://www.entrepreneur.com/article/362499,Entrepreneur,2021-01-25 13:00:00+00:00,How Millennials Are Changing Stock Investing,The thrifty nature of the nation s largest gen...,2021-01-25 13:30:00+00:00,2021-01-25
8,Danny Nelson,Crypto Miner Marathon Patent Group Buys $150M ...,Cryptocurrency mining company Marathon Patent ...,https://www.coindesk.com/miner-marathon-buys-b...,CoinDesk,2021-01-25 13:00:57+00:00,Crypto Miner Marathon Patent Group Buys 150M i...,Cryptocurrency mining company Marathon Patent ...,2021-01-25 13:30:00+00:00,2021-01-25
15,Benjamin Powers,Substack Newsletters Are Being Used to Spread ...,Scam Substack accounts impersonate various cry...,https://www.coindesk.com/index.php?p=569315,CoinDesk,2021-01-25 20:34:46+00:00,Substack Newsletters Are Being Used to Spread ...,Scam Substack accounts impersonate various cry...,2021-01-26 13:30:00+00:00,2021-01-26
